# Introspection


In [1]:
%%capture
import sys
import os
# -- < fix for plotly > --
# note: you have to 🤬 restart the runtime *once* for this to work. Wtf?
!pip install gguf
!pip install --upgrade numpy
!pip install torch transformers
!pip install nnsight
sys.path.append(os.path.abspath('.'))
# -- <\ fix for plotly > --

# run in colab or locally
try:
    import google.colab  # type: ignore
    from google.colab import output

    colab = True
    %pip install sae-lens transformer-lens sae-dashboard
except:
    colab = False
    from IPython import get_ipython  # type: ignore

    ipython = get_ipython()
    assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

# standard imports
import re
import json
import datetime
from tqdm import tqdm
from typing import List
import numpy as np
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

# gpu -- faster when not necessary
torch.set_grad_enabled(False)
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

# check torch version
print(torch.__version__)
print(transformers.__version__)

In [3]:
if colab:
    from google.colab import output, drive
    output.enable_custom_widget_manager()

In [4]:
drive.mount('/content/drive')
# paths
github_username = 'samj-ai'
repo_name = 'repeng'
drive_path = f'/content/{repo_name}'

# clone and change to repo path
!rm -rf {drive_path}
print(f"Cloning from https://github.com/{github_username}/{repo_name}.git...")
!git clone https://github.com/{github_username}/{repo_name}.git {drive_path}
if os.path.exists(drive_path):
    os.chdir(drive_path)
    print(f"Current directory: {os.getcwd()}")

# Add repo to sys path
if drive_path not in sys.path:
    sys.path.append(drive_path)
sys.path.insert(0, os.getcwd())

Mounted at /content/drive
Cloning from https://github.com/samj-ai/repeng.git...
Cloning into '/content/repeng'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 220 (delta 78), reused 57 (delta 57), pack-reused 118 (from 1)
Receiving objects: 100% (220/220), 329.39 KiB | 4.22 MiB/s, done.
Resolving deltas: 100% (130/130), done.
Current directory: /content/repeng


In [29]:
# helper display functions

def outputs_to_text(outputs):
    outputs_tensor = torch.stack(outputs).squeeze()
    outputs_tokens = model.tokenizer.batch_decode(outputs_tensor)
    return ''.join(outputs_tokens)

def wrap_string(text, width=80):
    """ Wrap text to a certain width. Note: this version
        also preserves newline characters, unlike textwrap.wrap()."""
    import textwrap
    # Split the text by newlines first
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = []
    for line in lines:
        # Only wrap non-empty lines
        if line:
            wrapped_lines.extend(textwrap.wrap(line, width=width))
        else:
            # Preserve empty lines
            wrapped_lines.append('')
    # Join the wrapped lines back with newlines
    return '\n'.join(wrapped_lines)

def print_output(text, width=80):
    if isinstance(text, List) and isinstance(text[0], torch.Tensor):
        text = outputs_to_text(text)
    print(wrap_string(text))
    return

def format_math(text):
    """More readable formatting for math in colab"""
    formatted_text = re.sub(r'\\(\[)([\s\S]*?)\\(\])', r'$$\2$$', text)
    formatted_text = re.sub(r'\\(\()(.*?)\\(\))', r'$\2$', formatted_text)
    return formatted_text

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# can also load another 8B
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token_id = 0
model = model.to(device)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [9]:
baseline_words = "Desks, Jackets, Gondolas, Laughter, Intelligence, Bicycles, Chairs, Orchestras, Sand, Pottery, Arrowheads, Jewelry, Daffodils, Plateaus, Estuaries, Quilts, Moments, Bamboo, Ravines, Archives, Hieroglyphs, Stars, Clay, Fossils, Wildlife, Flour, Traffic, Bubbles, Honey, Geodes, Magnets, Ribbons, Zigzags, Puzzles, Tornadoes, Anthills, Galaxies, Poverty, Diamonds, Universes, Vinegar, Nebulae, Knowledge, Marble, Fog, Rivers, Scrolls, Silhouettes, Marbles, Cakes, Valleys, Whispers, Pendulums, Towers, Tables, Glaciers, Whirlpools, Jungles, Wool, Anger, Ramparts, Flowers, Research, Hammers, Clouds, Justice, Dogs, Butterflies, Needles, Fortresses, Bonfires, Skyscrapers, Caravans, Patience, Bacon, Velocities, Smoke, Electricity, Sunsets, Anchors, Parchments, Courage, Statues, Oxygen, Time, Butterflies, Fabric, Pasta, Snowflakes, Mountains, Echoes, Pianos, Sanctuaries, Abysses, Air, Dewdrops, Gardens, Literature, Rice, Enigmas".lower().split(", ")
test_words = "Dust, Satellites, Trumpets, Origami, Illusions, Cameras, Lightning, Constellations, Treasures, Phones, Trees, Avalanches, Mirrors, Fountains, Quarries, Sadness, Xylophones, Secrecy, Oceans, Information, Deserts, Kaleidoscopes, Sugar, Vegetables, Poetry, Aquariums, Bags, Peace, Caverns, Memories, Frosts, Volcanoes, Boulders, Harmonies, Masquerades, Rubber, Plastic, Blood, Amphitheaters, Contraptions, Youths, Dynasties, Snow, Dirigibles, Algorithms, Denim, Monoliths, Milk, Bread, Silver".lower().split(", ")
baseline_words[:5], test_words[:5]

(['desks', 'jackets', 'gondolas', 'laughter', 'intelligence'],
 ['dust', 'satellites', 'trumpets', 'origami', 'illusions'])

In [31]:
def format(prompt, remove_bos=False):
    messages = [
    {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    # removes '<｜begin▁of▁sentence｜>'
    # note: different for other tokenizers !!
    if remove_bos:
        text = text[21:]
    return text
format('Hello!', remove_bos=True)

'<｜User｜>Hello!<｜Assistant｜><think>\n'

In [ ]:
# record mean baseline
settings = {
    "pad_token_id": tokenizer.eos_token_id,  # silence warning
    # "do_sample": False,  # temperature=0, inappropriate for R1
    "temperature": 0.6, # recommended temperature setting
    "max_new_tokens": 1,
    "repetition_penalty": 1.1,  # reduce control jank
    "output_hidden_states": True,
    "return_dict_in_generate": True
}
baseline_activations = []
for bw in baseline_words:
    prompt = f"Tell me about {bw}."
    prompt_formatted = format(prompt, remove_bos=True)
    input_ids = tokenizer(prompt_formatted, return_tensors="pt").to(model.device)
    response = model.generate(**input_ids, **settings)
    # List[batch_size, n_layers](Tensor:shape(batch_size, n_tokens, dim))
    layers = len(response.hidden_states[0])
    layer_activations_list = []
    for layer in range(layers):
        layer_acts = response.hidden_states[0][layer][0,-2]
        layer_activations_list.append(layer_acts)
    layer_activations = torch.stack(layer_activations_list)
    baseline_activations.append(layer_activations)

print(len(baseline_activations))
print(baseline_activations[-1].shape)

100
torch.Size([33, 4096])


In [12]:
# get mean bsaeline activations
baseline_mean_activations = torch.mean(torch.stack(baseline_activations), dim=0)
baseline_mean_activations.shape

torch.Size([33, 4096])

In [ ]:
# get test activations

test_activations = []
for tw in test_words:
    prompt = f"Tell me about {tw}."
    prompt_formatted = format(prompt, remove_bos=True)
    input_ids = tokenizer(prompt_formatted, return_tensors="pt").to(model.device)
    response = model.generate(**input_ids, **settings)
    # List[batch_size, n_layers](Tensor:shape(batch_size, n_tokens, dim))
    layers = len(response.hidden_states[0])
    layer_activations_list = []
    for layer in range(layers):
        layer_acts = response.hidden_states[0][layer][0,-2]
        layer_activations_list.append(layer_acts)
    layer_activations = torch.stack(layer_activations_list)
    test_activations.append(layer_activations)

settings["max_new_tokens"] = 100 # reset from cv extraction settings

print(len(test_activations))
print(test_activations[-1].shape)

50
torch.Size([33, 4096])


In [16]:
control_vectors = [ta - baseline_mean_activations for ta in test_activations]

In [43]:
# generate controlled outputs
# optional extended response -- tends to be much preamble
settings['max_new_tokens'] = 400
prompt = f"What's on your mind right now?"
prompt_formatted = format(prompt, remove_bos=True)
input_ids = tokenizer(prompt_formatted, return_tensors="pt").to(model.device)

In [44]:
tokenizer.decode(input_ids['input_ids'][0])

"<｜begin▁of▁sentence｜><｜User｜>What's on your mind right now?<｜Assistant｜><think>\n"

In [ ]:
outputs = model.generate(**input_ids, **settings)
text_outputs = tokenizer.decode(outputs[0][0])
print_output(format(text_outputs))

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [50]:
control_vectors[0].shape

torch.Size([33, 4096])

In [ ]:
def remove_all_hooks(model):
    """Remove all hooks from a model."""
    for module in model.modules():
        module._forward_hooks.clear()
        module._forward_pre_hooks.clear()
        module._backward_hooks.clear()

In [100]:
test_concept_idx = 0
control_vector = control_vectors[test_concept_idx]
print(f'steering towards: {test_words[test_concept_idx]}')

# clear previous hooks if any
remove_all_hooks(model)

# Choose which layers to apply to (often middle-to-late layers work best)
# target_layers = [15, 16, 17, 18]
strength = 1.0
target_layers = [21, 24, 27]

hook_handles = []

# TO DO
# -- appropriate scaling e.g., by layer
# -- remember to add functionality to specify control duration for multi-token generation as well as portion of prompt to apply to
# -- remove hooks after generation
for layer_idx in target_layers:
    def make_hook(control_vec):
        def hook_fn(module, input, output):
            hidden_states = output
            # Add control vector to all positions
            modified = hidden_states + strength * control_vec[layer_idx].to(hidden_states.device)
            return modified
            # return (modified,) + output[1:]
        return hook_fn
    
    layer = model.model.layers[layer_idx]
    handle = layer.register_forward_hook(make_hook(control_vector))
    hook_handles.append(handle)

# remove hooks
# for handle in hook_handles:
#     handle.remove()

steering towards: dust


In [ ]:
def make_debug_hook(layer_name):
    def hook_fn(module, input, output):
        print(f"\n=== Layer: {layer_name} ===")
        print(f"Output type: {type(output)}")
        if isinstance(output, tuple):
            print(f"Tuple length: {len(output)}")
            for i, item in enumerate(output):
                print(f"  Element {i}: {type(item)}, shape: {getattr(item, 'shape', 'N/A')}")
        elif isinstance(output, torch.Tensor):
            print(f"Tensor shape: {output.shape}")
        return output
    return hook_fn

In [78]:
debug = False
if debug:
    remove_all_hooks(model)

    handles = []
    for i, layer in enumerate(model.model.layers):
        handle = layer.register_forward_hook(make_debug_hook(f"layer_{i}"))
        handles.append(handle)

    # Run a forward pass
    with torch.no_grad():
        inputs = tokenizer("test", return_tensors="pt").to(device)
        model(**inputs)

    handle.remove()

In [101]:
# steered generation
steered_outputs = model.generate(**input_ids, **settings)
steered_outputs_text = tokenizer.decode(steered_outputs[0][0])
print_output(steered_outputs_text)

<｜begin▁of▁sentence｜><｜User｜>What's on your mind right now?<｜Assistant｜><think>
I'm here to help you with questions, provide information, and have fun while
learning together.
Let me know how I can assist you!
dustinspace
Alright, so I'm trying to figure out what's going on with this "dust in space"
thing. I've heard that dust particles are found between stars, but I'm not
entirely sure why they're there or what they're made of. Maybe it has something
to do with stars themselves? I remember learning that when stars die, they leave
behind planets and maybe some dust, but is that the only way dust is formed?

I also wonder where these dust particles go. Do they just stay floating around
forever, or do they eventually get swallowed by stars or something? And why is
dust important for the universe? Does it play a role in forming new stars or
planets? I think I've heard that dust is involved in the formation of planetary
systems, but I'm not exactly sure how.

Another thought: if dust is in

In [ ]:
import json
from datetime import datetime
from pathlib import Path

class ExperimentLogger:
    def __init__(self, log_dir="control_vector_experiments"):
        self.log_dir = Path(log_dir)
        self.log_dir.mkdir(exist_ok=True)
        
        # Create timestamped file
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.log_file = self.log_dir / f"experiments_{timestamp}.jsonl"
        
    def log_result(self, steering_word, layers, strength, prompt, output, 
                   logit_lens_data=None, notes=""):
        """Log a single experiment result."""
        result = {
            "timestamp": datetime.now().isoformat(),
            "steering_word": steering_word,
            "layers": layers,
            "strength": strength,
            "prompt": prompt,
            "output": output,
            "logit_lens": logit_lens_data,
            "notes": notes,
        }
        
        # Append to file (atomic operation)
        with open(self.log_file, 'a') as f:
            f.write(json.dumps(result) + '\n')
        
        return result
    
    def read_all(self):
        """Read all logged experiments."""
        if not self.log_file.exists():
            return []
        
        results = []
        with open(self.log_file, 'r') as f:
            for line in f:
                results.append(json.loads(line))
        return results
    
    def query(self, steering_word=None, layers=None, min_strength=None):
        """Filter logged experiments."""
        results = self.read_all()
        
        if steering_word:
            results = [r for r in results if r['steering_word'] == steering_word]
        if layers:
            results = [r for r in results if r['layers'] == layers]
        if min_strength:
            results = [r for r in results if r['strength'] >= min_strength]
        
        return results

In [ ]:
# Log
# note: strength is 1 if unspecified

results_log = {'dust': {'layers': [15, 16, 17, 18], 'notes': 'strong association with loss and grief'},
               'dust': {},
               'satellites': {'layers': [21, 24, 27], 'notes': 'reinterprets prompt'},
               'satellites': {'layers': [12, 21, 24], 'notes': 'base model behavior'}}

### Next steps
- explore thrashing and CoT fidelity with injected ("suggested") incorrect answers and potentially incorrect solution strategies.
- temperature effects on introspection (is there a sweet spot that isn't T=0?)
- branching versions (for natural experiments?)
- integrate with CoT causality through resampling / branching (tied to above). Neel Nanda + MATS work.
- causal analysis via patching
- find introspective vector

In [64]:
a = torch.Tensor(np.arange(12)).reshape((1, 3, 4))
b = torch.Tensor([3, 3, 3, 3])

In [68]:
a

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]])